# Oracles Tutorial

1) Motivation of the tutorial

2) Outline of the tutorial and what it sets out to accomplish

    Definitions of Quantum Oracles, including types of oracles
    
    Mathematical properties of Quantum Oracles
    https://github.com/microsoft/QuantumKatas/tree/main/tutorials/LinearAlgebra
    
    Demos of different quantum oracles
    
    phase kickback:
    https://quantumcomputing.stackexchange.com/questions/14206/should-c-not-gate-affect-input-qubit-or-not/14207#14207
    https://qiskit.org/textbook/ch-gates/phase-kickback.html
    
    Exercises to implement your own quantum oracles
    
    Final exercise to implement a more complex oracle as well as write tests for that oracle
    
3) Suggested tutorials or topics to be completed or familiar with before this tutorial – The Qubit (Tutorial), Linear Algebra (Tutorial), Basic Quantum Computing Gates (Tutorial), Superposition


# Part I - Introduction to Quantum Oracles

## Classical Oracles
In classical computing, the idea of white box versus black box testing is leveraged.  In white box testing, the implementation of the function, say to factor a number, is visable to the tester.  Thus the tester can test specific expectaions such as runtime or memory complexity.  However, in black box testing, the tester can only test the functionality and expected behavior of the function - the implementation has been abstracted away.

Generally, most functions and data structures that we use while programming are a black box in our eyes - we are not worried with how an linked list is implemented, we only care about the functionality that it provides.  

Suppose I provided you the following function:
the function takes two parameters as input, two lists, where these lists represent the availability during the week of two different employees at a company.  The function returns if true if there is a day (Monday, Tuesday, Wednesday, Thursday, or Friday) for which they are both free and could schedule a meeting, if no such date exists then the function returns false.

The previous proposition is an example of a **classical oracle** because when interacting with this function, you are not worried with *how* the function determines the answer, you only care about *what* the answer is when provided a specific input.

Formally, a **classical oracle** is a function for which when provided some input, it produces a specific *deterministic* output.  Hence the same input *always* results in the same output.

#### <span style="color:blue">Exercise 1</span>: Implement a classical oracle
Implement the following classical oracle:
* f(x) = 1 (True) if x = 7
* f(x) = 0 (False) otherwise
* provided that x is a binary string. Note, $x = 0101 = 5$, $x = 0110 = 6$, $x = 10001 = 17$

In [1]:
%kata E1_Classical_Oracle

function Is_Seven(x : String) : Bool {
    Message("Implement me!");
    return false;
}

C:\snippet_.qs(1,1): error QS3001: Syntax does not match any known patterns.


## Quantum Oracles

An oracle in the quantum world is a black box operation that is utilized during an algorithm.  These **quantum oracles** are passed as input to quantum algorithms and they themselves require inputs to operate on.  A quantum oracle implements some function $f: \{0,1\}^n \rightarrow \{0,1\}^m$ where $x$ is the input state of the form $|x\rangle = |x_0\rangle \otimes |x_1\rangle \otimes ... \otimes |x_{n-1}\rangle$ There are two types of quantum oracles: phase oracles and marking oracles.  Each of these types oracles are defined based on their operations on their basis states, where these basis states are always $|0\rangle$ and $|1\rangle$.  Oracles must be unitary as well as not change the input if that input is a basis state of the oracle.  Oracles also follow the same rules of linear algebra as normal matricies as they themselves are matricies, refer to the intro to review some of the properties of quantum operators (matricies).

### Phase Oracles
A phase oracle $U_{phase}$ is an oracle that when provided some state $|x\rangle$ it flips the sign of that state if $f(x)=1$.  Thus:

$$U_{phase} |x\rangle = (-1)^{f(|x\rangle)}|x\rangle$$

#### <span style="color:blue">Demo 1</span>: Simple phase quantum oracle - the AND oracle!
Consider the following phase oracle $U_{AND,phase}$:
* $U_{AND,phase} |x\rangle = (-1)^{f(x)}|x\rangle$
* $f(x) = 1$ if $|x\rangle = |111\rangle$
* $f(x) = 0$ otherwise
* $|x\rangle$ is composed of 3 qubits


In [2]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Diagnostics;

operation PhaseOracle_Demo () : Unit {
    let divider = "--------------------------------------------------------------------------------------------------";

    // allocate the |000⟩ state
    using(q = Qubit[3]) { 
        // Print out that we currently have the |000⟩ state
        // Notice that the phase of the state is positive
        Message("State |000⟩:");
        DumpMachine();
        Message(divider);

        // Convet the state to represet the number 4
        X(q[2]);

        // Print out that we currently have the |100⟩ state
        // Notice that the phase of the state is positive
        Message("State |100⟩:");
        DumpMachine();
        Message(divider);

        // Apply the AND phase oracle to to the state
        using(minus = Qubit()) {
            X(minus);
            Controlled Z(q, minus);
            Reset(minus);
        }
        // Print out that we currently have the |100⟩ state
        // Notice that the phase of the state is still positive
        Message("State |100⟩ after AND phase oracle:");
        DumpMachine();
        Message(divider);

        // Now lets switch to the state where f(x) = 1
        X(q[1]);
        X(q[0]);

        // Print out that we currently have the |111⟩ state
        // Notice that the phase of the state is positive
        Message("State |111⟩:");
        DumpMachine();
        Message(divider);

        // Let us apply the AND phase oracle
        using(minus = Qubit()) {
            X(minus);
            Controlled Z(q, minus);
            Reset(minus);
        }
        // Print out that we currently have the |111⟩ state
        // Notice that the phase of the state is now negative
        Message("State |111⟩ after AND phase oracle:");
        DumpMachine();
        Message(divider);
        
        // Return all of the qubits back to |0⟩ so they can be deallocated safely
        ResetAll(q);
    }
}

// focus is on the result -- check the qubit tutorial have the user
// just trust the implementation for now and in the next exercise they
// will convince themselves that it is correct

PhaseOracle_Demo

In [3]:
%simulate PhaseOracle_Demo

State |000⟩:


|0⟩	1 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0 + 0𝑖

--------------------------------------------------------------------------------------------------
State |100⟩:


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	1 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0 + 0𝑖

--------------------------------------------------------------------------------------------------
State |100⟩ after AND phase oracle:


...

|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	1 + 0𝑖

--------------------------------------------------------------------------------------------------
State |111⟩ after AND phase oracle:


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	-1 + 0𝑖

--------------------------------------------------------------------------------------------------


()

Now you will implement the same classical oracle that you implemented in <span style="color:blue">Exercise 1</span> as a quantum phase oracle.

#### <span style="color:blue">Exercise 2</span>: Implement a phase quantum oracle
Implement the following quantum oracle $U_{7,phase}$:
* $U_{7,phase} |x\rangle = (-1)^{f(x)}|x\rangle$
* $f(x) = 1$ if $x = 7$
* $f(x) = 0$ otherwise
* $|x\rangle$ is a state composted of $3$ qubits

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  What internal state can you prepare such that $X|\psi\rangle = -|\psi\rangle$ or $Z|\psi\rangle = -|\psi\rangle$?  Finally use this internal state to flip the sign of the input state $|x\rangle$.
</details>

In [4]:
%kata E2_Phase_Quantum_Oracle 

operation Phase_7_Oracle (x : Qubit[]) : Unit 
is Adj {
    Message("Implement me!");
}

C:\snippet_.qs(1,1): error QS3001: Syntax does not match any known patterns.


#### Mathematical properties

Consider how the oracle from <span style="color:blue">Exercise 2</span> acts on its basis states:
$$U_{7,phase} |111\rangle = -|111\rangle$$
$$U_{7,phase} |110\rangle = |110\rangle$$

This follows the requirement that a $U_{7,phase}$ does not change the input if it's a basis state as well as the fact that $U_{7,phase}$ does not change the norm of the state ($U_{7,phase}$ is unitary).  However, consider if we were provided a state in superposition instead, what might that look like?

Suppose that $|\beta\rangle$ is an equal super position of the $6$ and $7$ state: 
$$|\beta\rangle = \frac{1}{\sqrt{2}} \big(|110\rangle + |111\rangle\big) = |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|0\rangle + |1\rangle\big) = |11\rangle \otimes |+\rangle = |11+\rangle$$

Lets consider how our operator acts on this new state:
$$U_{7,phase} |\beta\rangle = U_{7,phase} \Big[\frac{1}{\sqrt{2}} \big(|110\rangle + |111\rangle\big)\Big] = \frac{1}{\sqrt{2}} \big(U_{7,phase} |110\rangle + U_{7,phase} |111\rangle\big) = \frac{1}{\sqrt{2}} \big(|110\rangle - |111\rangle\big) := |\gamma\rangle$$

Now the question is if our input state was modified during this operation, lets simplify $|\gamma\rangle$:
$$|\gamma\rangle = \frac{1}{\sqrt{2}} \big(|110\rangle - |111\rangle\big) = |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|0\rangle - |1\rangle\big) = |11\rangle \otimes |-\rangle = |11-\rangle \neq |\beta\rangle$$

Here we see that the oracle modifies the input.  This is conditioned on the input state being a *superposition* of the basis states of the oracle - as a phase oracle will only modify the sign of its basis states.

> It is also worth noting that while the oracle modified the input when provided a superposition state, it did *not* modify the norm of that state.  As an exercise, you can verify this yourself by taking the norm of $|\beta\rangle$ and $|\gamma\rangle$, which both will result in a value of $1$.
>
>
> As another exercise, consider how you could distinguish between the input and output state programatrically?  Is there an operation that you could apply to the initial state $|\beta\rangle$ and the final state $|\gamma\rangle$ to show that the two states are not equivalent?


### Marking Oracles

A marking oracle $U_{mark}$ is an oracle that when provided some state $|x\rangle$ and some qubit $y$ preforms addition modulo 2 between $f(x)$ and $y$.  Hence $U_{mark}$ is an operator that preforms the following operation:

$$U_{mark}|x\rangle |y\rangle = U_{mark}\big(|x\rangle \otimes |y\rangle\big) = |x\rangle \otimes |y \oplus f(x)\rangle = |x\rangle |y \oplus f(x)\rangle$$

#### <span style="color:blue">Demo 2</span>: Simple marking quantum oracle - the AND oracle!
Consider the following marking oracle $U_{AND,mark}$:
* $U_{AND,mark} |x\rangle|y\rangle = |x\rangle|y\oplus f(x)\rangle$
* $f(x) = 1$ if $x = |11...1\rangle$
* $f(x) = 0$ otherwise
* $|x\rangle$ is a state composted of $3$ qubits
* $|y\rangle$ is a single qubit


In [ ]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Diagnostics;

operation SimpleMarkingOracle_Demo() : Unit {
    let divider = "--------------------------------------------------------------------------------------------------";

    // allocate |y⟩
    using (y = Qubit()) {
        // Lets see the initial state of |y⟩
        Message("Initial |y⟩:");
        DumpMachine();
        Message(divider);
        
        // Lets see the result of feeding |000⟩ into our AND marking oracle with |y⟩ = |0⟩
        using (x = Qubit[3]) {
            Controlled X(x, y);
        }
        Message("Applying the AND oracle with |x⟩ = |000⟩, |y⟩ = |0⟩:");
        DumpMachine();
        Message(divider);
        
        // Lets see the result of feeding |100⟩ into our AND marking oracle with |y⟩ = |0⟩
        using (x = Qubit[3]) {
            X(x[2]);
            Controlled X(x, y);
            ResetAll(x);
        }
        Message("Applying the AND oracle with |x⟩ = |100⟩, |y⟩ = |0⟩:");
        DumpMachine();
        Message(divider);
        
        // Lets see the result of feeding |111⟩ into our AND marking oracle with |y⟩ = |0⟩
        using (x = Qubit[3]) {
            X(x[0]);
            X(x[1]);
            X(x[2]);
            Controlled X(x, y);
            ResetAll(x);
        }
        Message("Applying the AND oracle with |x⟩ = |111⟩, |y⟩ = |0⟩:");
        DumpMachine();
        Message(divider);
        
        // Lets see the result of feeding |011⟩ into our AND marking oracle with |y⟩ = |1⟩
        using (x = Qubit[3]) {
            X(x[0]);
            X(x[1]);
            Controlled X(x, y);
            ResetAll(x);
        }
        Message("Applying the AND oracle with |x⟩ = |011⟩, |y⟩ = |1⟩:");
        DumpMachine();
        Message(divider);
        
        Reset(y);
    }
}

In [ ]:
%simulate SimpleMarkingOracle_Demo

Now you will implement the same quantum oracle that you implemented in <span style="color:blue">Exercise 2</span> but this time as a marking oracle instead.

#### <span style="color:blue">Exercise 3</span>: Implement a marking quantum oracle
Implement the following quantum oracle $U_{7,mark}$:
* $U_{7,mark} |x\rangle |y\rangle = |x\rangle |y \oplus f(x)\rangle$
* $f(x) = 1$ if $x = 7$
* $f(x) = 0$ otherwise
* $|x\rangle$ is a state composted of $3$ qubits
* $|y\rangle$ is a single qubit

In [ ]:
%kata E3_Marking_Quantum_Oracle 

operation Marking_7_Oracle(x: Qubit[], y: Qubit) : Unit
is Adj {
    Message("Implement me!");
}

#### Mathematical Properties

Again, let us consider how the marking oracle that you just implemented in <span style="color:blue">Exercise 3</span> affects the input when the input is the oracles basis states opposed to a superposition state.
$$U_{7,mark} |111\rangle |0\rangle = |111\rangle |0 \oplus f(111)\rangle = |111\rangle |0 \oplus 1\rangle = |111\rangle |1\rangle$$
$$U_{7,mark} |111\rangle |1\rangle = |111\rangle |1 \oplus f(111)\rangle = |111\rangle |1 \oplus 1\rangle = |111\rangle |0\rangle$$

$$U_{7,mark} |110\rangle |0\rangle = |110\rangle |0 \oplus f(110)\rangle = |110\rangle |0 \oplus 0\rangle = |110\rangle |0\rangle$$
$$U_{7,mark} |110\rangle |1\rangle = |110\rangle |1 \oplus f(110)\rangle = |110\rangle |1 \oplus 0\rangle = |110\rangle |1\rangle$$

In the previous examples, all of the input $|x\rangle$ and $|y\rangle$ are in basis states of the oracle $U_{mark}^7$, however we see that even when providing input states, if $f(x)=1$ we will change the value of our input $|y\rangle$.  However, if $f(x)=0$ then neither the input $|x\rangle$ or $|y\rangle$ will be modified by the oracle.  By definition, if the marking oracle evaluates to one for some input, hence if $f(x)=1$ for some $x$, then the input state $|y\rangle$ will be changed.  Specifically, $|y\rangle$ will be flipped.

Now let us define a state $|\alpha\rangle$ such that $|x\rangle$ is a superposition of the $6$ and $7$ basis states and $|y\rangle = |0\rangle$:
$$|\alpha\rangle = \frac{1}{\sqrt{2}}\big(|110\rangle |0\rangle + |111\rangle |0\rangle\big) = |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|0\rangle + |1\rangle\big) \otimes |0\rangle = |11+\rangle |0\rangle$$

How does our marking oracle $U_{7,mark}$ act on our state $|\alpha\rangle$?
> Recall that oracles are simply operators and operators are linear thus they can be distrubuted to each term individually.

$$U_{7,mark} |\alpha\rangle = \frac{1}{\sqrt{2}} \big(U_{7,mark}|110\rangle |0\rangle + U_{7,mark}|111\rangle |0\rangle\big) = \frac{1}{\sqrt{2}} \big(|110\rangle |0\rangle + |111\rangle |1\rangle\big) := |\epsilon\rangle$$

Now we would like to observe how our input state $|\alpha\rangle$ was modified by the oracle.  Let's simplify the resulting state from the oracle, $|\epsilon\rangle$:
$$|\epsilon\rangle = \frac{1}{\sqrt{2}} \big(|110\rangle |0\rangle + |111\rangle |1\rangle\big) = |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|0\rangle |0\rangle + |1\rangle |1\rangle\big) = |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|00\rangle + |11\rangle\big) = |11\rangle \otimes |\Phi^+\rangle = |11\Phi^+\rangle$$

Here we see that we have now entangled our input states $|x\rangle$ and $|y\rangle$!  This is a common occurance for marking oracles when the input is a superposition of basis states: after applying the oracle, the input $|x\rangle$ will become entangled with $|y\rangle$.  Here we see that the result is the bell state $|\Phi^+\rangle$.

>As an exercise, what entangled state would we get in the previous example if $|y\rangle = |1\rangle$ instead of $|y\rangle = |0\rangle$?
>
> <br/>
> <details>
>   <summary><b>Click here for the answer!</b></summary>
>   The entangled state that is produced would be $|\Psi^+\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$.  Overall, we would get the following: $U_{mark}^7 |011+\rangle |1\rangle = |011\rangle |\Psi^+\rangle$
> </details>

How might the above derivation differ if both $|x\rangle$ and $|y\rangle$ are in superposition states?  We will consider this example once again with $|x\rangle$ as a superposition of the $6$ and $7$ states and we will now use $|y\rangle=|-\rangle$:
$$|\eta\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle + |111\rangle\big)\Big] \otimes \frac{1}{\sqrt{2}}\big(|0\rangle - |1\rangle\big) = \frac{1}{2} \big(|110\rangle|0\rangle + |111\rangle|0\rangle - |110\rangle|1\rangle - |111\rangle|1\rangle\big)$$

How does $U_{7,mark}$ act on our state $|\eta\rangle$?
$$U_{7,mark}|\eta\rangle = U_{7,mark} \frac{1}{2} \big(|110\rangle|0\rangle + |111\rangle|0\rangle - |110\rangle|1\rangle - |111\rangle|1\rangle \big)$$
$$= \frac{1}{2} \big( U_{7,mark}|110\rangle|0\rangle + U_{7,mark}|111\rangle|0\rangle - U_{7,mark}|110\rangle|1\rangle - U_{7,mark}|111\rangle|1\rangle \big) = \frac{1}{2} \big(|110\rangle|0\rangle + |111\rangle|1\rangle - |110\rangle|1\rangle - |111\rangle|0\rangle \big) := |\xi\rangle$$

Now we would like to observe how our input state $|\eta\rangle$ was modified by the oracle.  Let's simplify the resulting state from the oracle, $|\xi\rangle$:
$$|\xi\rangle = \frac{1}{2} \big(|110\rangle|0\rangle + |111\rangle|1\rangle - |110\rangle|1\rangle - |111\rangle|0\rangle\big) = \frac{1}{2} \big(|110\rangle|0\rangle - |110\rangle|1\rangle - |111\rangle|0\rangle + |111\rangle|1\rangle \big)$$
$$= \frac{1}{2} \Big[|110\rangle \otimes \big(|0\rangle - |1\rangle \big) + |111\rangle \otimes \big(|1\rangle - |0\rangle\big)\Big] = \Big[\frac{1}{\sqrt{2}} \big( |110\rangle - |111\rangle \big) \Big] \otimes \Big[ \frac{1}{\sqrt{2}} \big( |0\rangle - |1\rangle \big) \Big] = \Big[\frac{1}{\sqrt{2}} \big( |110\rangle - |111\rangle \big) \Big] \otimes |-\rangle$$

Now we can compare $|\eta\rangle$ and $|\xi\rangle$ directly, below we will repeat the final equations:
$$|\eta\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle + |111\rangle\big)\Big] \otimes |-\rangle$$
$$|\xi\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle - |111\rangle\big)\Big] \otimes |-\rangle$$

We can see that these two equations are identical except for the phase on the $|111\rangle$ basis state (representing $7$).  The result of this phenomena is refered to as **phase kickback** because the phase from $|y\rangle$ has "kicked back" and now appears in the $|x\rangle$ aswell.  Phase kickback will be discussed in more detail in the next section.

# Part II: Phase Kickback

As explored in the previous section, sometimes when providing input to an oracle, **phase kickback** will occur.  Phase kickback is a very important occurance and is used in plenty of quantum algorithms to compute a function.  Generally, there are operations like $X$ and $Z$ which preserve input states in their basis - up to a phase:
$$X|+\rangle=|+\rangle, \ X|-\rangle=-|-\rangle$$
$$Z|0\rangle=|0\rangle, \ Z|1\rangle=-|1\rangle$$

When we begin to consider controlled operations is when phase kickback comes more into play.  So far in this tutorial, you have implemented two controlled operaitons - $U_{7,phase}$ and $U_{7,mark}$.  These operations are considered controlled because the form of the output state is *controlled* by the form of the input state.  Specifically, for $U_{7,phase}$ we implemented that when $|x\rangle=|111\rangle$ then a global phase would be applied to the output state.  Likewise with $U_{7,mark}$ we *controlled* the value of the input state $|y\rangle$ based on the value of the input state $|x\rangle$ to produce some output state.  As we saw in the most recent example for $U_{7,mark}$ where we had an input state $|x\rangle=\frac{1}{\sqrt{2}} \big( |110\rangle + |111\rangle \big)$ and $|y\rangle = |-\rangle$ phase kickback occured.  Phase kickback generally only occurs when your control state *and* input state are both superpositions states - it occurs in other situations as well but this is the one situation where the kickback of the phase is *observable*.  Recall that we can only observe our states via measurement, DumpMachine is a useful tool for us to see the states but really the states are acting in a probabalistic mannor.

Say that we wanted to see if phase kickback had occured.  Lets take the previous example with $U_{7,mark}$ and see how we could determine wether or not phase kickback occured.
$$|\eta\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle + |111\rangle\big)\Big] \otimes |-\rangle = |11\rangle \otimes \frac{1}{\sqrt{2}}\big( |0\rangle + |1\rangle \big) = |11+\rangle$$
$$|\xi\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle - |111\rangle\big)\Big] \otimes |-\rangle = |11\rangle \otimes \frac{1}{\sqrt{2}}\big( |0\rangle - |1\rangle \big) = |11-\rangle$$

<br/>
<details>
  <summary><b>Now how could we differentiate between the state $|11+\rangle$ and $|11-\rangle$?  Take a moment to think, then click here to see if you were correct!</b></summary>
  
    That's right!  If we apply a Hadamard operator to the third qubit, we will be able to distinguish between the input state and the output state.
    
    $$(I\otimes I \otimes H)|11+\rangle = |110\rangle$$
    $$(I\otimes I \otimes H)|11-\rangle = |111\rangle$$
    
    Now if we were to measure the input state versus the output state we would detect that the phase from $|y\rangle=|-\rangle$ was kicked back into our input state $|x\rangle$!
</details>

One important application of phase kickback is that it allows us to convert a marking oracle into a phase oracle.

#### <span style="color:blue">Exercise 4</span>: Implement an oracle converter
* **Input:** a marking oracle
* **Output:** a phase oracle that takes a state $|x\rangle$ and flips the phase of that state subject to the condition of the marking oracle.

> A oracle converter is useful because many quantum algorithms rely on a phase oracle, such as Grover's algorithm, however it is often easier to implement a marking oracle.  This converter will provide us a way to convert a marking oracle we are interested in into a phase oracle; which could then be leveraged in a quantum algorithm such as Grover's algorithm.

<br/>
<details>
  <summary><b>If you're stuck on where to begin, click here!</b></summary>
    Recall that you can allocate extra qubits to assist in any operation.  Is there a state that you could prepare with an auxiliary qubit which would enable you to flip the phase of the input state $|x\rangle$ subject to the function $f(x)$ for the marking oracle?  Also recall that qubits can only be allocated in operations, not in functions.
</details>

In [ ]:
%kata E4_Oracle_Converter

function Oracle_Converter(markingOracle: ((Qubit[], Qubit) => Unit is Adj)) : (Qubit[] => Unit is Adj) {
    Message("Implement me!");
    return ConstructPhaseOracle(markingOracle, _);
}

operation ConstructPhaseOracle(markingOracle: ((Qubit[], Qubit) => Unit is Adj), qubits: Qubit[]) : Unit
is Adj {
    using (minus = Qubit()) {
        within {
            X(minus);
            H(minus);
        } apply {
            markingOracle(qubits, minus);
        }
    }
}

# Part III: Implementing Quantum Oracles

In this section you will implement a few quantum oracles of your own - each representing a specified function.  If you observe the operation declarations below, you will see that there is a requirement for the adjoint of the operation to be valid - the operation preformed in reverse order must be valid.  This is common practice: when we write quantum oracles, we want to undo any operations we may have performed on the input to achive the result in order to ensure that the oracle does not change the input if that input is a basis state of the oracle.

#### <span style="color:blue">Exercise 5</span>: Implement the alternating oracle
Implement the following quantum oracle $U_{alt,1}$:
* $U_{alt,1} |x\rangle |y\rangle = |x\rangle |y \oplus f(x)\rangle$
* $f(x) = 1$ if $|x\rangle=|101010...10\rangle$ OR $|x\rangle=|101010...01\rangle$
* $f(x) = 0$ otherwise
* $|x\rangle$ is a state composted of $n$ qubits
* $|y\rangle$ is a single qubit

<br/>
<details>
  <summary><b>Before implementing this oracle, answer the following question: are you implmenting a marking or a phase oracle?  Click here for the answer!</b></summary>
    Correct!  This is a marking oracle because we are flipping the value of some target qubit $|y\rangle$ based on the value of the other input $|x\rangle$
</details>

In [ ]:
$kata E5_Alternating_1_Oracle

operation Alternating_1_Oracle(x: Qubit[], y: Qubit) : Unit
is Adj {
    Message("Implement me!");
}

#### <span style="color:blue">Exercise 6</span>: Implement the kth spin-up oracle:
Implement the following quantum oracle $U_{k,spinup}$:
* $U_{k,spinup} |x\rangle = (-1)^{f(x)}|x\rangle$
* $f(x) = 1$ if the $k$th qubit in $|x\rangle$ is a $1$ $\big(\text{if } |x\rangle_k=|1\rangle \big)$
* $f(x) = 0$ otherwise
* $|x\rangle$ is a state composted of $n$ qubits
* $k$ is an integer with $k\in \{0, ..., n-1 \}$

<br/>
<details>
  <summary><b>Before implementing this oracle, answer the following question: are you implmenting a marking or a phase oracle?  Click here for the answer!</b></summary>
    Correct!  This is a phase oracle because we are applying a phase to the input state $|x\rangle$ based on the evaluation of the function $f(x)$.
</details>

> Can you implement this function without using auxiliary qubits?

In [6]:
%kata E6_kth_Spin_Up

operation kth_Spin_Up(x: Qubit[], k: Int) : Unit 
is Adj {
    Message("Implement me!");
}

C:\snippet_.qs(1,1): error QS3001: Syntax does not match any known patterns.


As you noticed in the previous exericse, undoing operations preformed on the input is essential to ensure that we are implementing oracles that follow the requirements of an oracle - oracles are unitary and they do not change the input if that input is a basis state.  Another key tool to have when implementing quantum oracles is allocation of auxiliary qubits to assist in a computation.  Below are a some exercises where you will practice allocating extra qubits to assist with the computation of $f(x)$. 

#### <span style="color:blue">Exercise 7</span>: Implement the alternating oracle
Implement the following quantum oracle $U_{alt,2}$:
* $U_{alt,2} |x\rangle = (-1)^{f(x)}|x\rangle$
* $f(x) = 1$ if $|x\rangle=|101010...10\rangle$ OR $|x\rangle=|101010...01\rangle$
* $f(x) = 0$ otherwise
* $|x\rangle$ is a state composted of $n$ qubits

<br/>
<details>
  <summary><b>Before implementing this oracle, answer the following question: are you implmenting a marking or a phase oracle?  Click here for the answer!</b></summary>
    Correct!  This is a phase oracle because we are applying a phase to the input state $|x\rangle$ based on the evaluation of the function $f(x)$.
</details>

In [ ]:
$kata E7_Alternating_2_Oracle

operation Alternating_2_Oracle(x: Qubit[]) : Unit
is Adj {
    Message("Implement me!");
}

#### <span style="color:blue">Exercise 8</span>: Implement the OR oracle
Implement the following quantum oracle $U_{or}$:
* $U_{or} |x\rangle |y\rangle = |x\rangle |y \oplus f(x)\rangle$
* $f(x) = 0$ if $|x\rangle=|000...0\rangle$ (all zeros)
* $f(x) = 1$ otherwise
* $|x\rangle$ is a state composted of $n$ qubits
* $|y\rangle$ is a single qubit

<br/>
<details>
  <summary><b>Before implementing this oracle, answer the following question: are you implmenting a marking or a phase oracle?  Click here for the answer!</b></summary>
    Correct!  This is a marking oracle because we are flipping the value of some target qubit $|y\rangle$ based on the value of the other input $|x\rangle$
</details>

In [ ]:
$kata E8_Or_Oracle

operation Or_Oracle(x: Qubit[], y: Qubit) : Unit
is Adj {
    Message("Implement me!");
}

# Part IV: More Oracles!  Implementation and Testing

teach them to allocate aux qubits for computation

In [ ]:
// write test cases for the alternating phase oracle
// compare this implementation with the oracle produced when 
// providing the marking oracle as input to the oracle converter 
// so the learner can convince themselves of their oracle converter
// implementation

In [ ]:
// exercise to determine if two people can meet during the week for a meeting (SAT oracle)

In [ ]:
// exercise to generalize previous result to see if m people can meet during a 2 week timeframe

# Part V: What's next?